## Atualização Cadastral - Veículos

Este notebook realiza um web scraping no site placafipe.com ou keplaca.com para extrair informações do veículo a partir da placa. As informações incluem: Marca, Modelo, Importado, Ano, Ano Modelo, Cor, Cilindrada, Combustível, Motor, Passageiros, UF, Município, Segmento, FIPE. Após isso, as informações são atualizadas em uma tabela desejada.
Caso seja usado em projetos reias é sugerido que troque o web scraping por a API: apiplacas.com.br.

In [ ]:
# importação dos pacotes
import pandas as pd
import requests
from bs4 import BeautifulSoup

# definição do cabeçalho HTTP
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
}


In [ ]:
# Converter listas para dicionairos usanod o separador : 
def lista_para_dicionario(lista):
    dicionario = {}
    for item in lista:
        if ':' in item:
            chave, valor = item.split(':', 1) 
            dicionario[chave.strip()] = valor.strip() 
    return dicionario

def buscar_veiculo(veiculo):
    url = f"https://placafipe.com/placa/{veiculo}" # site onde será realizado o web scraping 
    # url =  f"https://www.keplaca.com/chassi/{veiculo}"também pode ser buscado por chassi sem grandes alterações
    r = requests.get(url, headers=headers)
    if r != 200: # site alternativo
        url = f"https://www.keplaca.com/placa?placa-fipe={veiculo}"
        r = requests.get(url, headers=headers)
    print(r.status_code)

    soup = BeautifulSoup(r.text, 'lxml')
   # Filtro das tabelas de interesse: informações do veículo e valor da FIPE.
    tables_details = soup.find_all("table", class_='fipeTablePriceDetail')[0] 
    tables_fipe = soup.find_all("table", class_='fipe-mobile')[0]

    table1 = tables_details.find_all('tr')
    tr1 = []  
    for i in table1:
        a = i.text.strip()
        tr1.append(a)
    tr1 = lista_para_dicionario(tr1)
    tr1.pop('Chassi')
    table2 = tables_fipe.find_all('tr')
    tr2 = []  
    for i in table2:
        a = i.text.strip()
        tr2.append(a)
    tr2 = lista_para_dicionario(tr2)
    tr1['Fipe'] = tr2['Valor']
    return tr1, tr2  # retornando informações desejadas no dicionário tr1: Marca,Modelo,Importado,Ano,Ano Modelo,Cor,Cilindrada,Combustível,Motor,Passageiros,UF,Município,Segmento,Fipe 

def preencher_dados(dataset, placa):
    indice = dataset.query(f"Placa == '{placa}'").index[0]
    line_copy = dataset.iloc[indice].copy()
    infos, fipe = buscar_veiculo(placa)
    line_copy.update(pd.Series(infos))
    dataset.iloc[indice] = line_copy

In [ ]:
data = pd.read_csv('dataset.csv').astype(object)

In [ ]:
data

In [ ]:
for i in data['Placa']:
    preencher_dados(data, i)

data